In [ ]:
!pip install optuna

In [175]:
import torch
from torchvision import datasets, transforms
from torch import nn
import optuna
import numpy as np
from optuna.pruners import ThresholdPruner
import torch.nn.functional as F

In [176]:
criterion = nn.CrossEntropyLoss()
class CifarNet(nn.Module):
    def __init__(self):
        super(CifarNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 64, 5)
        self.conv4 = nn.Conv2d(64, 128, 3)
        self.conv5 = nn.Conv2d(128, 128, 3)
        self.conv6 = nn.Conv2d(128, 256, 3)
        self.conv7 = nn.Conv2d(256, 256, 3)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(36864, 512)
        self.fc2 = nn.Linear(512, 128)
        self.dropout = nn.Dropout(p=0.3)
        self.fc3 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.flat(x)
        # print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)        
        x = self.fc3(x)
        return x

In [177]:
def train(trial,num_epochs,optimizer,training_generator,validation_generator,model,device):
    train_accuracy = []
    valid_accuracy = []
    for epoch in range(num_epochs):
        model.train()
        running_accuracy = []
        for local_batch, local_labels in training_generator:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
            optimizer.zero_grad()
            outputs = model(local_batch)
            loss = criterion(outputs, local_labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs, 1)
            running_accuracy.append((predicted == local_labels).sum().item()/training_generator.batch_size)
        # Compute Training Accuracy
        train_accuracy.append(np.mean(running_accuracy))
        model.eval()
        running_accuracy = []
        with torch.no_grad():
            for local_batch, local_labels in validation_generator:
                local_batch, local_labels = local_batch.to(device), local_labels.to(device)
                outputs = model(local_batch)
                _, predicted = torch.max(outputs, 1)
                running_accuracy.append((predicted == local_labels).sum().item()/training_generator.batch_size)
        valid_accuracy.append(np.mean(running_accuracy))
        print('[{:d}] | train accuracy {:5.2f} | validation accuracy {:5.2f}'.format(epoch + 1,train_accuracy[-1]*100,valid_accuracy[-1]*100))
        # Study Parameters
        if len(valid_accuracy) > 4:
            trial.report(np.var(valid_accuracy[-5:]), epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()
    return valid_accuracy[-1]

In [178]:
data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

train_ds = datasets.CIFAR10(root ='data/',
                            train=True,
                            download=True,
                            transform=data_transform)

val_ds = datasets.CIFAR10(root ='data/',
                          train=False,
                          download=True,
                          transform=data_transform)

Files already downloaded and verified
Files already downloaded and verified


In [180]:
def objective(trial):
    device = torch.device("cuda:0")
    learning_rate = trial.suggest_uniform("lr", 0.0001, 0.005)
    batch_size = trial.suggest_categorical('batch_size',[16, 32, 64])
    num_epochs = trial.suggest_categorical('num_epoch',[30,50,70])
    training_generator = torch.utils.data.DataLoader(train_ds,
                                             batch_size=batch_size,
                                              shuffle=True)
    validation_generator = torch.utils.data.DataLoader(val_ds,
                                             batch_size=batch_size,
                                              shuffle=True)
    model = CifarNet()
    model.to(device)
    optimizer = torch.optim.RMSprop(model.parameters(),lr=learning_rate)
    print(f'lr {learning_rate} \t batch size {batch_size} \t epochs {num_epochs}')
    intermediate_value=train(trial,
                             num_epochs,
                             optimizer,
                             training_generator,
                             validation_generator,
                             model,
                             device)

    return intermediate_value

In [181]:
study = optuna.create_study(pruner=ThresholdPruner(lower=0.001),study_name='StudyCifar10.v1')
study.optimize(objective, n_trials=50)

[I 2021-02-02 09:40:46,638] A new study created in memory with name: StudyCifar10.v1


lr 0.0008224072901753402 	 batch size 16 	 epochs 30
[1] | train accuracy 25.99 | validation accuracy 36.94
[2] | train accuracy 41.60 | validation accuracy 45.95
[3] | train accuracy 47.96 | validation accuracy 50.47
[4] | train accuracy 52.60 | validation accuracy 53.04
[5] | train accuracy 55.64 | validation accuracy 56.72
[6] | train accuracy 57.99 | validation accuracy 57.21


[I 2021-02-02 09:47:42,573] Trial 0 pruned. 


[7] | train accuracy 58.98 | validation accuracy 55.66
lr 0.002795108174238946 	 batch size 32 	 epochs 50
[1] | train accuracy  9.82 | validation accuracy  9.98
[2] | train accuracy  9.83 | validation accuracy  9.98
[3] | train accuracy  9.62 | validation accuracy  9.98
[4] | train accuracy  9.88 | validation accuracy  9.98


[I 2021-02-02 09:50:59,855] Trial 1 pruned. 


...

[5] | train accuracy  9.61 | validation accuracy  9.95
lr 0.00266683661870935 	 batch size 64 	 epochs 70
[1] | train accuracy 20.44 | validation accuracy 25.05
[2] | train accuracy 18.23 | validation accuracy  9.95
[3] | train accuracy  9.88 | validation accuracy  9.95
[4] | train accuracy  9.74 | validation accuracy  9.95
[5] | train accuracy  9.80 | validation accuracy  9.95


[I 2021-02-02 13:34:31,307] Trial 48 pruned. 


[6] | train accuracy  9.96 | validation accuracy  9.95
lr 0.0019109806809108959 	 batch size 64 	 epochs 50
[1] | train accuracy  9.89 | validation accuracy  9.95
[2] | train accuracy  9.80 | validation accuracy  9.95
[3] | train accuracy  9.93 | validation accuracy  9.95
[4] | train accuracy  9.80 | validation accuracy  9.95


[I 2021-02-02 13:37:05,108] Trial 49 pruned. 


[5] | train accuracy  9.84 | validation accuracy  9.95


In [182]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_lr,params_num_epoch,state
0,0,6.383720e-04,2021-02-02 09:40:46.641518,2021-02-02 09:47:42.573412,0 days 00:06:55.931894,16,0.000822,30,PRUNED
1,1,0.000000e+00,2021-02-02 09:47:42.574333,2021-02-02 09:50:59.855900,0 days 00:03:17.281567,32,0.002795,50,PRUNED
2,2,0.000000e+00,2021-02-02 09:50:59.856847,2021-02-02 09:56:52.297398,0 days 00:05:52.440551,16,0.002225,30,PRUNED
3,3,0.000000e+00,2021-02-02 09:56:52.298821,2021-02-02 10:01:44.227261,0 days 00:04:51.928440,16,0.002739,70,PRUNED
4,4,0.000000e+00,2021-02-02 10:01:44.228838,2021-02-02 10:04:57.608039,0 days 00:03:13.379201,32,0.002807,50,PRUNED
5,5,0.000000e+00,2021-02-02 10:04:57.609395,2021-02-02 10:08:51.688044,0 days 00:03:54.078649,32,0.002610,70,PRUNED
6,6,0.000000e+00,2021-02-02 10:08:51.689012,2021-02-02 10:13:45.545729,0 days 00:04:53.856717,16,0.002021,70,PRUNED
7,7,0.000000e+00,2021-02-02 10:13:45.546596,2021-02-02 10:16:59.508462,0 days 00:03:13.961866,32,0.003743,50,PRUNED
8,8,0.000000e+00,2021-02-02 10:16:59.509280,2021-02-02 10:19:32.782728,0 days 00:02:33.273448,64,0.000628,30,PRUNED
9,9,0.000000e+00,2021-02-02 10:19:32.783921,2021-02-02 10:22:46.307482,0 days 00:03:13.523561,32,0.000925,30,PRUNED
